##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker를 사용한 텍스트 분류

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/lite/models/modify/model_maker/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/lite/models/modify/model_maker/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/lite/models/modify/model_maker/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

TensorFlow Lite Model Maker 라이브러리를 사용하면 기기 내 ML 애플리케이션용으로 TensorFlow 모델을 배포할 때 모델을 조정하고 특정 입력 데이터로 변환하는 프로세스가 단순해집니다.

이 노트북은 모바일 장치에서 영화 리뷰를 분류하는 데 목적을 두고 일반적으로 사용되는 텍스트 분류 모델을 적용하고 변환하는 작업을 설명하기 위해 Model Maker 라이브러리를 활용하는 엔드 투 엔드 예제를 보여줍니다. 텍스트 분류 모델은 텍스트를 미리 정의된 범주로 분류합니다. 입력은 사전 처리된 텍스트여야 하고 출력은 범주의 확률입니다. 이 튜토리얼에서 사용된 데이터세트는 긍정적이고 부정적인 영화 리뷰입니다.

## 전제 조건


### 필수 패키지 설치하기

이 예제를 실행하려면 [GitHub 리포지토리](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)의 Model Maker 패키지를 포함하여 필요한 패키지를 설치합니다.

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker-nightly

필요한 패키지를 가져옵니다.

In [ ]:
import numpy as np
import os

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

### 샘플 훈련 데이터를 다운로드합니다.

이 튜토리얼에서는 [GLUE](https://nlp.stanford.edu/sentiment/index.html) 벤치마크의 작업 중 하나인 [SST-2](https://gluebenchmark.com/)(Stanford Sentiment Treebank)를 사용합니다. 여기에는 훈련용 영화 리뷰 67,349개와 테스트용 영화 리뷰 872개가 포함되어 있습니다. 데이터세트에는 긍정적인 영화 리뷰와 부정적인 영화 리뷰의 두 가지 클래스가 있습니다.

In [ ]:
data_dir = tf.keras.utils.get_file(
      fname='SST-2.zip',
      origin='https://dl.fbaipublicfiles.com/glue/data/SST-2.zip',
      extract=True)
data_dir = os.path.join(os.path.dirname(data_dir), 'SST-2')

SST-2 데이터세트는 TSV 형식으로 저장됩니다. TSV와 CSV의 유일한 차이점은 TSV가 CSV 형식에서 쉼표 `,` 대신 탭 `\t` 문자를 구분 기호로 사용한다는 것입니다.

다음은 훈련 데이터세트의 처음 5줄입니다. label=0은 부정적, label=1은 긍정적을 의미합니다.

문장 | 레이블 |  |  |
--- | --- | --- | --- | ---
hide new secretions from the parental units | 0 |  |  |
contains no wit , only labored gags | 0 |  |  |
that loves its characters and communicates something rather beautiful about human nature | 1 |  |  |
remains utterly satisfied to remain the same throughout | 0 |  |  |
on the worst revenge-of-the-nerds clichés the filmmakers could dredge up | 0 |  |  |

다음으로 데이터세트를 Pandas 데이터 프레임에 로드하고 현재 레이블 이름(`0` 및 `1`)을 좀 더 사람이 읽을 수 있는 이름(`negative` 및 `positive`)으로 변경하여 모델 훈련에 사용합니다.


In [ ]:
import pandas as pd

def replace_label(original_file, new_file):
  # Load the original file to pandas. We need to specify the separator as
  # '\t' as the training data is stored in TSV format
  df = pd.read_csv(original_file, sep='\t')

  # Define how we want to change the label name
  label_map = {0: 'negative', 1: 'positive'}

  # Excute the label change
  df.replace({'label': label_map}, inplace=True)

  # Write the updated dataset to a new file
  df.to_csv(new_file)

# Replace the label name for both the training and test dataset. Then write the
# updated CSV dataset to the current folder.
replace_label(os.path.join(os.path.join(data_dir, 'train.tsv')), 'train.csv')
replace_label(os.path.join(os.path.join(data_dir, 'dev.tsv')), 'dev.csv')

## 빠른 시작

텍스트 분류 모델을 학습시키는 5단계는 다음과 같습니다.

**1단계. 텍스트 분류 모델 아키텍처를 선택합니다.**

여기서 우리는 적절한 정확도로 작고 빠른 모델을 생성하는 평균 단어 임베딩 모델 아키텍처를 사용합니다.

In [ ]:
spec = model_spec.get('average_word_vec')

Model Maker는 [BERT](https://arxiv.org/abs/1810.04805)와 같은 다른 모델 아키텍처도 지원합니다. 다른 아키텍처에 대해 알아보려면 아래의 [텍스트 분류기에 대한 모델 아키텍처 선택](#scrollTo=kJ_B8fMDOhMR) 섹션을 참조하세요.

**2단계. 훈련 및 테스트 데이터를 로드한 다음 특정 `model_spec`에 따라 전처리합니다.**

Model Maker는 CSV 형식의 입력 데이터를 가져올 수 있습니다. 이전에 생성한 사람이 읽을 수 있는 레이블 이름으로 훈련 및 테스트 데이터세트를 로드합니다.

각 모델 아키텍처는 입력 데이터를 특정한 방식으로 처리해야 합니다. `DataLoader`는 `model_spec`으로부터 요구 사항을 읽고 필요한 사전 처리를 자동으로 실행합니다.

In [ ]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=spec,
      is_training=False)

**3단계. 훈련 데이터로 TensorFlow 모델을 훈련합니다.**

평균 단어 임베딩 모델은 기본적으로 `batch_size = 32`를 사용합니다. 따라서 훈련 데이터세트에서 67,349개의 문장을 살펴보는 데 2104단계가 필요함을 알 수 있습니다. 우리는 10 epoch 동안 모델을 훈련할 것입니다. 이는 훈련 데이터세트를 10번 거치는 것을 의미합니다.

In [ ]:
model = text_classifier.create(train_data, model_spec=spec, epochs=10)

**4단계. 테스트 데이터로 모델을 평가합니다.**

훈련 데이터세트의 문장을 사용하여 텍스트 분류 모델을 훈련한 후 테스트 데이터세트의 나머지 872개 문장을 사용하여 이전에 본 적이 없는 새로운 데이터에 대해 모델이 어떻게 수행되는지 평가합니다.

기본 배치 크기는 32이므로 테스트 데이터세트의 872개 문장을 살펴보는 데 28단계가 필요합니다.

In [ ]:
loss, acc = model.evaluate(test_data)

**5단계. TensorFlow Lite 모델로 내보냅니다.**

TensorFlow Lite 형식으로 학습시킨 텍스트 분류를 내보내 보겠습니다. 모델을 내보낼 폴더를 지정합니다. 기본적으로, float TFLite 모델은 평균 단어 임베딩 모델 아키텍처에 대해 내보내집니다.

In [ ]:
model.export(export_dir='average_word_vec')

Colab의 왼쪽 사이드바를 사용하여 TensorFlow Lite 모델 파일을 다운로드할 수 있습니다. 위의 `export_dir` 매개변수에 지정된 `average_word_vec` 폴더로 이동하여 `model.tflite` 파일을 마우스 오른쪽 버튼으로 클릭하고 `Download`를 선택하여 로컬 컴퓨터에 다운로드합니다.

이 모델은 [TensorFlow Lite 작업 라이브러리](https://www.tensorflow.org/lite/inference_with_metadata/task_library/nl_classifier)의 [NLClassifier API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview)를 사용하여 Android 또는 iOS 앱에 통합할 수 있습니다.

모델이 작동하는 앱에서 사용되는 방식에 대한 자세한 내용은 [TFLite 텍스트 분류 샘플 앱](https://github.com/tensorflow/examples/blob/master/lite/examples/text_classification/android/lib_task_api/src/main/java/org/tensorflow/lite/examples/textclassification/client/TextClassificationClient.java#L54)을 참조하세요.

*참고 1: Android Studio 모델 바인딩은 아직 텍스트 분류를 지원하지 않으므로 TensorFlow Lite 작업 라이브러리를 사용하세요.*

*참고 2: TFLite 모델과 동일한 폴더에 `model.json` 파일이 있습니다. 여기에는 TensorFlow Lite 모델 내부에 번들로 제공되는 [메타데이터](https://www.tensorflow.org/lite/models/convert/metadata)의 JSON 표현이 포함됩니다. 모델 메타데이터는 TFLite 작업 라이브러리가 모델이 하는 역할과 모델에 대한 데이터를 사전/사후 처리하는 방식을 알도록 도와줍니다. `model.json` 파일은 참조 목적이고 그 내용이 이미 TFLite 파일 내에 들어 있으므로 다운로드할 필요가 없습니다.*

*참고 3: MobileBERT 또는 BERT-Base 아키텍처를 사용하여 텍스트 분류 모델을 훈련하는 경우, 대신 [BertNLClassifier API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier)를 사용하여 훈련된 모델을 모바일 앱에 통합해야 합니다.*

다음 섹션에서는 자세한 내용을 보여주기 위해 단계별로 예제를 안내합니다.

## 텍스트 분류기의 모델 아키텍처 선택하기

각 `model_spec` 객체는 텍스트 분류기에 대한 특정 모델을 나타냅니다. TensorFlow Lite Model Maker는 현재 [MobileBERT](https://arxiv.org/pdf/2004.02984.pdf), 평균 단어 임베딩 및 [BERT-Base](https://arxiv.org/pdf/1810.04805.pdf) 모델을 지원합니다.

지원 모델 | model_spec의 이름 | 모델 설명 | 모델 크기
--- | --- | --- | ---
평균 단어 임베딩 | 'average_word_vec' | RELU 활성화로 평균 텍스트 단어 임베딩. | &lt;1MB
MobileBERT | 'mobilebert_classifier' | BERT-Base보다 4.3배 작고 5.5배 빠르며 온디바이스 애플리케이션에 적합한 경쟁력 있는 결과를 달성합니다. | 양자화 시 25MB<br> 양자와 없을 시 100MB
<a>BERT 기반</a> | 'bert_classifier' | NLP 작업에서 널리 사용되는 표준 BERT 모델입니다. | 300MB

빠른 시작에서는 평균 단어 임베딩 모델을 사용했습니다. 더 높은 정확도로 모델을 훈련하기 위해 [MobileBERT](https://arxiv.org/pdf/2004.02984.pdf)로 전환해 보겠습니다.

In [ ]:
mb_spec = model_spec.get('mobilebert_classifier')

## 훈련 데이터 로드하기

이 튜토리얼을 진행하기 위해 자신의 데이터세트를 업로드할 수 있습니다. Colab의 왼쪽 사이드바를 사용하여 데이터세트를 업로드하세요.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_text_classification.png" width="800" hspace="100" alt="파일 업로드">

데이터세트를 클라우드에 업로드하지 않으려면, [가이드](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)에 따라 로컬에서 라이브러리를 실행할 수도 있습니다.

간단하게 하기 위해 이전에 다운로드한 SST-2 데이터 세트를 재사용합니다. `DataLoader.from_csv` 메서드를 사용하여 데이터를 로드해 보겠습니다.

모델 아키텍처가 변경됨에 따라 새로운 사전 처리 논리를 적용하려면 훈련 및 테스트 데이터세트를 다시 로드해야 합니다.

In [ ]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=False)

Model Maker 라이브러리는 데이터를 로드하는 `from_folder()` 메서드도 지원합니다. 같은 클래스의 텍스트 데이터가 같은 하위 디렉토리에 있고 하위 폴더 이름이 클래스 이름이라고 가정합니다. 각 텍스트 파일에는 하나의 영화 리뷰 샘플이 포함되어 있습니다. `class_labels` 매개변수는 하위 폴더를 지정하는 데 사용됩니다.

## TensorFlow 모델 학습하기

훈련 데이터를 사용하여 텍스트 분류 모델을 훈련시킵니다.

*참고: MobileBERT는 복잡한 모델이므로 Colab GPU에서 각 훈련 epoch에는 약 10분이 소요됩니다. GPU 런타임을 사용하고 있는지 확인하세요.*

In [ ]:
model = text_classifier.create(train_data, model_spec=mb_spec, epochs=3)

자세한 모델 구조를 조사합니다.

In [ ]:
model.summary()

## 모델 평가하기

테스트 데이터를 사용하여 방금 훈련한 모델을 평가하고 손실 및 정확도 값을 측정합니다.

In [ ]:
loss, acc = model.evaluate(test_data)

## TensorFlow Lite 모델로 내보내기

나중에 장치 내 ML 애플리케이션에서 사용할 수 있도록 훈련된 모델을 [메타데이터](https://www.tensorflow.org/lite/models/convert/metadata)가 포함된 TensorFlow Lite 모델 형식으로 변환합니다. 레이블 파일과 어휘 파일은 메타데이터에 포함됩니다. 기본 TFLite 파일 이름은 `model.tflite`입니다.

많은 기기 내 ML 애플리케이션에서 모델 크기는 중요한 요소입니다. 따라서 모델을 더 작게 만들고 잠재적으로 더 빠르게 실행하려면 양자화를 적용하는 것이 좋습니다. 기본 훈련 후 양자화 기술은 BERT 및 MobileBERT 모델에 대한 동적 범위 양자화입니다.

In [ ]:
model.export(export_dir='mobilebert/')

TensorFlow Lite 모델 파일은 [TensorFlow Lite 작업 라이브러리](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier)의 [BertNLClassifier API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview)를 사용하여 모바일 앱에 통합될 수 있습니다. 이것은 평균 단어 벡터 모델 아키텍처로 훈련된 텍스트 분류를 통합하는 데 사용되는 <code>NLClassifier</code> API와 <strong>다릅니다</strong>.

내보내기 형식은 다음 중 하나 또는 목록일 수 있습니다.

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.VOCAB`
- `ExportFormat.SAVED_MODEL`

기본적으로, 모델 메타데이터가 포함된 TensorFlow Lite 모델 파일만 내보냅니다. 더 나은 검사를 위해 모델과 관련된 다른 파일을 내보내도록 선택할 수도 있습니다. 예를 들어 다음과 같이 레이블 파일과 어휘 파일만 내보냅니다.

In [ ]:
model.export(export_dir='mobilebert/', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])

`evaluate_tflite` 메서드로 TFLite 모델을 평가하여 정확도를 측정할 수 있습니다. 훈련된 TensorFlow 모델을 TFLite 형식으로 변환하고 양자화를 적용하면 정확도에 영향을 줄 수 있으므로 배포 전에 TFLite 모델 정확도를 평가하는 것이 좋습니다.

In [ ]:
accuracy = model.evaluate_tflite('mobilebert/model.tflite', test_data)
print('TFLite model accuracy: ', accuracy)

## 고급 사용법

`create` 함수는 Model Maker 라이브러리가 모델을 생성하는 데 사용하는 드라이버 함수입니다. `model_spec` 매개변수는 모델 사양을 정의합니다. `AverageWordVecSpec` 및 `BertClassifierSpec` 클래스가 현재 지원됩니다. `create` 함수는 다음 단계로 구성됩니다.

1. `model_spec`에 따라 텍스트 분류자의 모델을 만듭니다.
2. 분류자 모델을 훈련합니다 기본 epoch 및 기본 배치 크기는 `model_spec` 객체에서 `default_training_epochs` 및 `default_batch_size` 변수로 설정합니다.

이 섹션에서는 모델 조정 및 하이퍼 매개변수 훈련과 같은 고급 사용법 주제를 다룹니다.

### MobileBERT 모델 하이퍼파라미터 사용자 지정하기

조정할 수 있는 모델 매개변수는 다음과 같습니다.

- `seq_len`: 모델에 공급할 시퀀스의 길이입니다.
- `initializer_range`: 모든 가중치 행렬을 초기화하기 위한 truncated_normal_initializer의 표준 편차입니다.
- `trainable`: 사전 훈련된 레이어가 훈련 가능한지 여부를 지정하는 boolean입니다.

조정할 수 있는 훈련 파이프라인 매개변수는 다음과 같습니다.

- `model_dir`: 모델 체크포인트 파일의 위치입니다. 설정하지 않으면 임시 디렉토리가 사용됩니다.
- `dropout_rate`: 드롭아웃 비율입니다.
- `learning_rate`: Adam 옵티마이저의 초기 학습률입니다.
- `tpu`: 연결할 TPU 주소입니다.

예를 들어 `seq_len=256`(기본값은 128)으로 설정할 수 있습니다. 이렇게 하면 모델이 더 긴 텍스트를 분류할 수 있습니다.

In [ ]:
new_model_spec = model_spec.get('mobilebert_classifier')
new_model_spec.seq_len = 256

### 평균 단어 임베딩 모델 하이퍼파라미터 사용자 지정하기

`AverageWordVecSpec` 클래스의 `wordvec_dim` 및 `seq_len` 변수와 같은 모델 인프라를 조정할 수 있습니다.


예를 들어, `wordvec_dim`의 더 큰 값을 사용하여 모델을 훈련할 수 있습니다. 모델을 수정하는 경우 새 `model_spec`을 구성해야 합니다.

In [ ]:
new_model_spec = AverageWordVecSpec(wordvec_dim=32)

전처리된 데이터를 가져옵니다.

In [ ]:
new_train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=new_model_spec,
      is_training=True)

새 모델을 훈련합니다.

In [ ]:
model = text_classifier.create(new_train_data, model_spec=new_model_spec)

### 훈련 하이퍼 매개변수 조정하기

모델 정확성에 영향을 미치는 `epochs` 및 `batch_size`와 같은 훈련 하이퍼 매개변수를 조정할 수도 있습니다. 예를 들어, 다음과 같습니다.

- `epochs`: epoch가 많을수록 정확성이 향상될 수 있지만, 과대적합으로 이어질 수 있습니다.
- `batch_size`: 하나의 훈련 단계에서 사용할 샘플의 수입니다.

예를 들어, 더 많은 epoch에서 훈련할 수 있습니다.

In [ ]:
model = text_classifier.create(new_train_data, model_spec=new_model_spec, epochs=20)

20개의 훈련 epoch에서 새로 재훈련된 모델을 평가합니다.

In [ ]:
new_test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=new_model_spec,
      is_training=False)

loss, accuracy = model.evaluate(new_test_data)

### 모델 아키텍처 변경하기

`model_spec`을 변경하여 모델을 변경할 수 있습니다. 다음은 BERT-Base 모델로 변경하는 방법을 보여줍니다.

`model_spec`을 텍스트 분류자에 대한 BERT-Base 모델로 변경합니다.

In [ ]:
spec = model_spec.get('bert_classifier')

나머지 단계는 같습니다.

### TensorFlow Lite 모델에서 훈련 후 양자화 사용자 정의하기

[훈련 후 양자화](https://www.tensorflow.org/lite/performance/post_training_quantization)는 모델 정확도를 약간만 떨어트리면서 모델 크기와 추론 지연 시간을 줄이는 동시에 CPU 및 하드웨어 가속기의 추론 속도도 개선할 수 있는 변환 기술입니다. 따라서 모델을 최적화하는 데 널리 사용됩니다.

Model Maker 라이브러리는 모델을 내보낼 때 기본 훈련 후 양자화 기술을 적용합니다. 훈련 후 양자화를 사용자 지정하려는 경우 Model Maker는 [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig)를 사용하여 여러 훈련 후 양자화 옵션도 지원합니다. float16 양자화를 예로 들어보겠습니다. 먼저 양자화 구성을 정의합니다.

```python
config = QuantizationConfig.for_float16()
```

그런 다음 이러한 구성을 가진 TensorFlow Lite 모델을 내보냅니다.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# 더 읽어보기

기술적인 세부 사항을 알아보려면 [텍스트 분류](https://www.tensorflow.org/lite/examples/text_classification/overview) 예제를 읽어볼 수 있습니다. 자세한 내용은 다음을 참조하세요.

- TensorFlow Lite Model Maker [가이드](https://www.tensorflow.org/lite/models/modify/model_maker) 및 [API 참조](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker)
- 작업 라이브러리: 배포용 [NLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/nl_classifier) 및 [BertNLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier)
- 엔드 투 엔드 참조 앱: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/text_classification/android) 및 [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/text_classification/ios)